In [ ]:

!pip install qiskit qiskit-machine-learning

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report
from qiskit.circuit.library import ZZFeatureMap
from qiskit.utils import QuantumInstance
from qiskit import Aer
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.algorithms import QSVC

df = pd.read_csv("synthetic_pki_dataset.csv")
X = df.drop("label", axis=1).values
y = df["label"].values
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

feature_map = ZZFeatureMap(feature_dimension=X_train.shape[1], reps=2)
quantum_instance = QuantumInstance(Aer.get_backend('qasm_simulator'), shots=1024)
kernel = QuantumKernel(feature_map=feature_map, quantum_instance=quantum_instance)
qsvc = QSVC(quantum_kernel=kernel)
qsvc.fit(X_train, y_train)

y_pred = qsvc.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
